In [2]:
import numpy as np
import pandas as pd
import json
import re
import MeCab
from collections import Counter

/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/var/pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def read_jasonl(filename):
    with open(filename) as f:
        return [json.loads(line.rstrip('\r\n')) for line in f.readlines()]

In [4]:
flatten = lambda l: [item for sublist in l for item in sublist if len(item) is not 0]

In [4]:
with open("../data/compound_train.json", 'r') as f:
    train = json.load(f)

In [5]:
wiki_data = read_jasonl("../data/jawiki-cirrussearch-dump_of_Compound.jsonl")

## カテゴリリスト作成

In [69]:
def remove_disambiguation_word(title):
    title_ = title.replace(")", ")").replace("(", "(")
    m = re.match(r'([^\s]+)\s*\((化合物|化学)\)', title_)
    if m:
        return m.group(1)
    return title

In [12]:
category_list = flatten([entry['category'] for entry in wiki_data])

In [75]:
compound_list = [remove_disambiguation_word(entry['title']) for entry in wiki_data]

In [79]:
category_df = pd.DataFrame(Counter(category_list).most_common()).rename(columns={0: "category", 1: "count_"})
category_df.head()

,category,count_
0,化学関連のスタブ項目,1795
1,ISBNマジックリンクを使用しているページ,826
2,薬学関連のスタブ項目,379
3,J-Global IDが未設定になっている記事,301
4,アルコール,242


In [80]:
# 化合物名は除去して保存
category_df.loc[~category_df.category.isin(compound_list)].to_csv("../data/category_list.csv", index=False)

In [6]:
Youto_dict = {}
for entry in train['entry']:
    Youto_dict[str(entry['WikipediaID'])] = entry['Attributes']['用途']

In [7]:
train_entry = [entry for entry in wiki_data for train_entry in train['entry'] if entry['index']['_id'] == str(train_entry['WikipediaID'])]

In [13]:
pd.DataFrame(
    {"category": list(set([category for t in train_entry for category in t['category'] if is_contain(category, Youto_dict[t['index']['_id']])]))}
).to_csv("../data/category_in_train_data.csv", index=False)

In [1]:
def is_contain(category, youto_list):
    youto = ','.join(youto_list)
    
    return re.search(category, youto)